In [1]:
import pandas as pd
import numpy as np
import pyodbc
import sys
import json
import urllib
import sqlalchemy
import os
import shapely
from datetime import datetime

sys.path.append('../')
#from IPM_Shared_Code_public.Python.geo_functions import read_geosql
from IPM_Shared_Code_public.Python.utils import get_config
from IPM_Shared_Code_public.Python.email_functions import get_contacts, read_template, send_email
from geosupport import *
from get_address_points import *

In [20]:
pd.set_option('display.max_columns', None)

In [2]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
parksgis = config['srv']['parksgis']
data_parks_server = config['srv']['data_parks']
structuresdb = config['db']['structuresdb']
portal = config['url']['portal']
structures_url = config['url']['structures']
geosupport_url = config['url']['geosupport']

In [3]:
geo_key = config['keys']['geosupport_key']
geo_ip = config['keys']['geosupport_ip']

In [4]:
params = urllib.parse.quote_plus(r'Driver=' + driver + ';Server=' + server +
                                  ';Database=' + structuresdb +
                                  ';Trusted_Connection=Yes;')
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
connection = engine.connect()

In [5]:
sql = 'select * from structuresdb.dbo.tbl_parks_structures where bin is not null and bin != 0 and bin % 1000000 > 1'

In [6]:
structs = pd.read_sql(con=engine, sql=sql)

In [7]:
final_df = master_geosupport_func2(structs)

In [22]:
final_df

,bin,high_address_number,low_address_number,street_name,out_boro_name1,address_id,posted_address,b7sc,tpad_bin_status,type,type_meaning,out_bbl,out_TPAD_bin,out_TPAD_bin_status,out_TPAD_conflict_flag,out_error_message,out_grc,in_bin,official_address,addressable,out_zip_code,out_hurricane_zone,out_co,out_cd,out_sd,out_nta,out_ad,out_com_dist,out_fire_bat,out_fire_co,out_fire_co_str,out_fire_div,out_b10sc1,out_police_patrol_boro,out_police_area,out_police_pct,out_san_sched,out_san_dist_section,out_san_recycle,out_san_reg,out_san_org_pick_up,out_usps_city_name,out_preferred_lgc,out_sos_ind,out_physical_id
0,3340691,9000,9000,BAY PARKWAY,BROOKLYN,3217154,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-99,NaN,NaN,Addressable,11214,1,43,11,23,BK27 / Bath Beach,46,311,43,Ladder 166,Ladder 166,8,31733001010,Brooklyn South,,62,,311 / 11,,,,BROOKLYN,31733001,Address is on the right when facing from BELT ...,0051350
1,3081931,694,694,THOMAS S BOYLAND STREET,BROOKLYN,3109361,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-99,NaN,NaN,Addressable,11212,6,41,09,19,BK81 / Brownsville,55,316,44,Engine 231,Engine 231,15,35083001020,Brooklyn North,,73,2A,316 / 162,EF,MWF,,BROOKLYN,35083001,Address is on the right when facing from DUMON...,0071115
2,3423681,986,986,WASHINGTON AVENUE,BROOKLYN,10170962,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-99,NaN,NaN,Addressable,11225,X,35,09,21,BK99 / park-cemetery-etc-Brooklyn,57,355,38,Engine 280,Engine 280,15,38973001010,Brooklyn South,,78,1A,355 / 551,,,,BROOKLYN,38973001,Address is on the right when facing from PRESI...,0038250
3,3031049,145,145,BROOKLYN AVENUE,BROOKLYN,3038223,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-99,NaN,NaN,Addressable,11213,X,36,09,25,BK61 / Crown Heights North,43,308,38,Ladder 132,Ladder 132,15,32333001010,Brooklyn North,,77,2A,308 / 082,EW,MWF,,BROOKLYN,32333001,Address is on the left when facing from ST MAR...,0049023
4,3031309,175,175,BROOKLYN AVENUE,BROOKLYN,5198901,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-99,NaN,NaN,Addressable,11213,X,36,09,25,BK61 / Crown Heights North,43,308,38,Ladder 132,Ladder 132,15,32333001010,Brooklyn North,,77,2A,308 / 082,EW,MWF,,BROOKLYN,32333001,Address is on the left when facing from PROSPE...,0049022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3742,2086438,,,5 BOROUGH SHOPS,BRONX,NaN,NaN,20080501,N/A,N,Non-Addressable Place Name,2025890031,N/A,N/A,N/A,,00,2086438,1.0,Addressable,10454,2,17,15,29,BX39 / Mott Haven-Port Morris,84,201,14,Ladder 29,Ladder 29,6,20080501010,Bronx,,40,2A,201 / 012,EF,MWF,,BRONX,20080501,Address is on the right when facing from AMTRA...,0057280
3743,2086438,850,850,EAST 138 STREET,BRONX,2007364,NaN,22580001,N/A,,Ordinary Address Range,2025890031,N/A,N/A,N/A,,00,2086438,1.0,Addressable,10454,2,17,15,29,BX39 / Mott Haven-Port Morris,84,201,14,Ladder 29,Ladder 29,6,22580001010,Bronx,,40,2A,201 / 012,EF,MWF,,BRONX,22580001,Address is on the right when facing from AMTRA...,0057280
3744,2086438,231,231,WALNUT AVENUE,BRONX,NaN,NaN,27265001,N/A,,Ordinary Address Range,2025890031,N/A,N/A,N/A,,00,2086438,1.0,Addressable,10454,2,17,15,29,BX39 / Mott Haven-Port Morris,84,201,14,Ladder 29,Ladder 29,6,27265001010,Bronx,,40,2A,201 / 012,EF,MWF,,BRONX,27265001,Address is on the left when facing from EAST ...,0059659
3745,2086438,,,EAST 137 STREET,BRONX,NaN,NaN,22577001,N/A,W,Non-Addressable Building Frontage,2025890031,N/A,N/A,N/A,,00,2086438,1.0,Addressable,,,,,,/,,,,,,,22577001010,,,,,/,,,,,225770,,


In [23]:
final_df[(final_df['posted_address'] == 1) & (final_df['official_address'] == 1)]

,bin,high_address_number,low_address_number,street_name,out_boro_name1,address_id,posted_address,b7sc,tpad_bin_status,type,type_meaning,out_bbl,out_TPAD_bin,out_TPAD_bin_status,out_TPAD_conflict_flag,out_error_message,out_grc,in_bin,official_address,addressable,out_zip_code,out_hurricane_zone,out_co,out_cd,out_sd,out_nta,out_ad,out_com_dist,out_fire_bat,out_fire_co,out_fire_co_str,out_fire_div,out_b10sc1,out_police_patrol_boro,out_police_area,out_police_pct,out_san_sched,out_san_dist_section,out_san_recycle,out_san_reg,out_san_org_pick_up,out_usps_city_name,out_preferred_lgc,out_sos_ind,out_physical_id


In [27]:
def check_address_type(df):
    if (df['low_address_number'] == df['high_address_number'] and 
        df['low_address_number'] == ''):
        
        if np.isnan(df['address_id']):
            address_type = 'Official and Posted Single Range Address'
        
        else:
            address_type = 'Official Only Single Range Address'
            
    else:

        if df['low_address_number'] == '':
            address_type = 'Common Place Address'

        else:
            address_type = 'Ranged Address'
        

    return address_type
        
            

In [25]:
final_df['low_address_number']

0       9000
1        694
2        986
3        145
4        175
        ... 
3742        
3743     850
3744     231
3745        
3746     747
Name: low_address_number, Length: 3747, dtype: object

In [29]:
final_df['address_type_check'] = final_df.apply(lambda x: check_address_type(x), axis = 1)

In [31]:
final_df['address_type_check']

0                                 Ranged Address
1                                 Ranged Address
2                                 Ranged Address
3                                 Ranged Address
4                                 Ranged Address
                          ...                   
3742    Official and Posted Single Range Address
3743                              Ranged Address
3744                              Ranged Address
3745    Official and Posted Single Range Address
3746                              Ranged Address
Name: address_type_check, Length: 3747, dtype: object

In [15]:
final_df[final_df['address_id'].notnull()][['bin', 'low_address_number', 'high_address_number', 'street_name', 'address_id']]

,bin,low_address_number,high_address_number,street_name,address_id
0,3340691,9000,9000,BAY PARKWAY,3217154
1,3081931,694,694,THOMAS S BOYLAND STREET,3109361
2,3423681,986,986,WASHINGTON AVENUE,10170962
3,3031049,145,145,BROOKLYN AVENUE,3038223
4,3031309,175,175,BROOKLYN AVENUE,5198901
...,...,...,...,...,...
3720,2122201,130,130,REYNOLDS AVENUE,5170179
3726,2117523,811,811,EAGLE AVENUE,9117991
3730,2118219,500,500,EAST 178 STREET,5151368
3740,2118281,851,851,HORNADAY PLACE,5151312
